In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt 
import random
import urllib
import cv2

from keras.models import Sequential

from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from tensorflow.keras import layers

from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import LeakyReLU

import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import shutil

import matplotlib.pyplot as plt
import numpy as np
import itertools

from keras.layers import Lambda

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = "/content/drive/My Drive/colab/dataset_bonus"

In [ ]:
img_height = 224
img_width  = 224
classes    = ['0','1','2','3','4']
batch_size = 50

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle="true")

Found 35126 files belonging to 5 classes.
Using 24589 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle="true")

Found 35126 files belonging to 5 classes.
Using 7025 files for validation.


In [ ]:
num_classes = 5

min=-1
max=1

filepath = "weights.best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=True, 
                             mode='auto', save_freq='epoch')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=50, 
                      verbose=1, mode='auto')


from keras.applications.resnet50 import ResNet50
model = ResNet50()

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=500,
  callbacks=[checkpoint, early]
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=123, 
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle="true",
  color_mode="grayscale")

model.load_weights("weights.best.hdf5")
score = model.evaluate(test_ds)
print('Test score: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
model.summary()

In [ ]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

predictions = np.array([])
true_labels =  np.array([])
for x, y in test_ds:
  predictions = np.concatenate([predictions, model.predict_classes(x)])
  true_labels = np.concatenate([true_labels, y.numpy()])

precisions, recall, f1_score, samples_per_class = metrics.precision_recall_fscore_support(true_labels, predictions,labels = [0,1,2,3,4,5,6,7,8,9])
confusion_matrix = metrics.confusion_matrix(true_labels, predictions)
print('Precision :',precisions,'\nRecall :',recall,'\nF1_Score :',f1_score,'\nSamples per class :',samples_per_class)

In [ ]:
target_names=['0','1','2','3','4','5','6','7','8','9']
# plot_confusion_matrix(confusion_matrix, target_names, title='Confusion matrix', cmap=None, normalize=None)
accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix).astype('float')
misclass = 1 - accuracy
cmap = plt.get_cmap('Blues')

plt.figure(figsize=(8, 6))
plt.imshow(confusion_matrix, interpolation='nearest', cmap=cmap)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(target_names))
plt.xticks(tick_marks, target_names, rotation=45)
plt.yticks(tick_marks, target_names)

thresh = confusion_matrix.max() / 2
for i, j in itertools.product(range(confusion_matrix.shape[0]), range(confusion_matrix.shape[1])):
    plt.text(j, i, "{:,}".format(confusion_matrix[i, j]),
              horizontalalignment="center",
              color="white" if confusion_matrix[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('Labels')
plt.xlabel('Predicted Labels\naccuracy: {:0.4f} - misclass: {:0.4f}'.format(accuracy, misclass))
plt.show()

In [ ]:
epochs = range(0,len(history.history['loss']))
loss_train = history.history['loss']
loss_val   = history.history['val_loss']
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'r', label='Validation loss')
plt.title('Training/Validation losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
epochs = range(0,len(history.history['accuracy']))
acc_train = history.history['accuracy']
acc_val   = history.history['val_accuracy']
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'r', label='Validation accuracy')
plt.title('Training/Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()